# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
import random, math
import os
from IPython.display import display, clear_output

El ejecutable se corre indicando el archivo y un valor $p \in (0, 1)$

In [2]:
def correr_experimento(p_value, archivo_instancia):
    # Leer archivo de la instancia.
    # instancia = leer_instancia(archivo_instancia
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./page_rank", archivo_instancia, str(p_value)], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    # process.stdin.write(instancia)
    # process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()
    
    print(exit_code)

    # Verificar que el proceso no fallo.
    if exit_code != 0: print(F"Hubo un error en la experimentacion con la instancia {archivo_instancia}")

    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();

In [3]:
def agregar_experimentos(p_value, dataframe):
    counter_n = 1
    while counter_n < 50:
        for n in range(0, 30):
            fila_n = dataframe.iloc[n];
            if counter_n < 50:
                experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["M"], p_value, fila_n["archivo"]])
                counter_n = counter_n + 1

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [4]:
gaussean={}
casos = ["random", "densas", "ralas"]
experimentos = []

casos_pvalue = ["random-random", "densas-pbajo", "ralas-pbajo", "densas-palto", "ralas-palto"]

for caso in casos:
    gaussean[caso]=pd.read_csv(F"instancias/{caso}/indice.csv")
    
for caso_p in casos_pvalue:
    for caso_c in casos:
        if caso_p.split("-")[1] == 'random':
            p_value = random.uniform(0, 1)
            agregar_experimentos(p_value, gaussean[caso_c])
            
        #if caso_p.split("-")[1] == 'pbajo':
            #p_value = random.uniform(0.5, 1)
            #agregar_experimentos(p_value, gaussean[caso_c])   
            
        #if caso_p.split("-")[1] == 'palto':
            #p_value = random.uniform(0, 0.5)
            #agregar_experimentos(p_value, gaussean[caso_c])
    
if(not os.path.isdir("resultados")):
    os.mkdir("resultados")
    
print(experimentos[1])

FileNotFoundError: [Errno 2] No such file or directory: 'instancias/densas/indice.csv'

De todas las instancias, tomamos las que tienen 20 o mas negocios para poder efectuar un analisis entre tiempos comparables. En caso contrario, se colapsan y no se obtiene mucha información.

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [12]:
columnas = ["dataset", "n", "M", "p", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    display('Matrix Size: ' + str(experimento[1]))
    display('Relationship: ' + str(experimento[2]))
    display('P value: ' + str(experimento[3])) 
    display('Path: ' + str(experimento[4]))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    for i in range(0, T):
        correr_experimento(experimento[3], experimento[4])
        
#df_resultado = pd.DataFrame(filas, columns=columnas);
#df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 147/147'

'Matrix Size: 30'

'Relationship: 33'

'P value: 0.08315180114731946'

'Path: instancias/ralas/Test-ralas-19.txt'

3221225781
Hubo un error en la experimentacion con la instancia instancias/ralas/Test-ralas-19.txt
3221225781
Hubo un error en la experimentacion con la instancia instancias/ralas/Test-ralas-19.txt
3221225781
Hubo un error en la experimentacion con la instancia instancias/ralas/Test-ralas-19.txt
3221225781
Hubo un error en la experimentacion con la instancia instancias/ralas/Test-ralas-19.txt
3221225781
Hubo un error en la experimentacion con la instancia instancias/ralas/Test-ralas-19.txt
